In [1]:
import subprocess
import os
import sys
import pandas as pd
import glob

In [2]:
folder_name = "data/us_mails_more"
output_folder = "data/us_mails_more/emails"

In [3]:
df = pd.read_csv("data/us_mails_more/IT Companies_data.csv")
df

,Name,Website
0,Jafton | Mobile App Development New York,https://www.jafton.com/top-mobile-app-developm...
1,RisingMax - IT Consulting Company,https://risingmax.com/
2,IN NYC IT,http://innycit.com/
3,Exigent Technologies | NYC Managed IT Services,https://www.exigent.net/contact/office-locatio...
4,Agicent App Company NYC,https://www.agicent.com/
...,...,...
54494,Vandis,https://www.vandis.com/
54495,"New York Gourmet Coffee, LLC",http://www.newyorkgourmetcoffee.com/
54496,Oilex Fuel of New York,https://oilexfuelli.com/
54497,Mineola Fuel Ltd,https://www.longislandcod.com/


In [4]:
from urllib.parse import urlparse
df['homepage'] = df['Website'].apply(lambda x: '://'.join(urlparse(x)[0:2]))
df['domain'] = df['homepage'].apply(lambda x: urlparse(x).netloc)
df

,Name,Website,homepage,domain
0,Jafton | Mobile App Development New York,https://www.jafton.com/top-mobile-app-developm...,https://www.jafton.com,www.jafton.com
1,RisingMax - IT Consulting Company,https://risingmax.com/,https://risingmax.com,risingmax.com
2,IN NYC IT,http://innycit.com/,http://innycit.com,innycit.com
3,Exigent Technologies | NYC Managed IT Services,https://www.exigent.net/contact/office-locatio...,https://www.exigent.net,www.exigent.net
4,Agicent App Company NYC,https://www.agicent.com/,https://www.agicent.com,www.agicent.com
...,...,...,...,...
54494,Vandis,https://www.vandis.com/,https://www.vandis.com,www.vandis.com
54495,"New York Gourmet Coffee, LLC",http://www.newyorkgourmetcoffee.com/,http://www.newyorkgourmetcoffee.com,www.newyorkgourmetcoffee.com
54496,Oilex Fuel of New York,https://oilexfuelli.com/,https://oilexfuelli.com,oilexfuelli.com
54497,Mineola Fuel Ltd,https://www.longislandcod.com/,https://www.longislandcod.com,www.longislandcod.com


In [5]:
df.nunique()

Name        54499
Website     54499
homepage    51352
domain      51005
dtype: int64

In [6]:
# drop the duplicates in homepage column
df = df.drop_duplicates(subset=['homepage'])
df

,Name,Website,homepage,domain
0,Jafton | Mobile App Development New York,https://www.jafton.com/top-mobile-app-developm...,https://www.jafton.com,www.jafton.com
1,RisingMax - IT Consulting Company,https://risingmax.com/,https://risingmax.com,risingmax.com
2,IN NYC IT,http://innycit.com/,http://innycit.com,innycit.com
3,Exigent Technologies | NYC Managed IT Services,https://www.exigent.net/contact/office-locatio...,https://www.exigent.net,www.exigent.net
4,Agicent App Company NYC,https://www.agicent.com/,https://www.agicent.com,www.agicent.com
...,...,...,...,...
54494,Vandis,https://www.vandis.com/,https://www.vandis.com,www.vandis.com
54495,"New York Gourmet Coffee, LLC",http://www.newyorkgourmetcoffee.com/,http://www.newyorkgourmetcoffee.com,www.newyorkgourmetcoffee.com
54496,Oilex Fuel of New York,https://oilexfuelli.com/,https://oilexfuelli.com,oilexfuelli.com
54497,Mineola Fuel Ltd,https://www.longislandcod.com/,https://www.longislandcod.com,www.longislandcod.com


In [7]:
# Partition in 30 csv files
n = 30
partition = len(df) // n
for i in range(n):
    df.iloc[i*partition:(i+1)*partition].to_csv(f"{folder_name}/domains/domains_{i}.csv", index=False)
    

In [11]:
def run_crawl(start_url_csv, folder_name,i):
    subprocess.Popen(f'''cd {os.getcwd()} && \
                    scrapy crawl fetch_emails_us \
                    -a start_url_csv="{start_url_csv}" \
                    -o "{folder_name}/emails/emails_{i}.csv" \
                    --logfile "{folder_name}/logs/log_{i}.txt"''',
                    shell=True)

In [12]:
csv_files = glob.glob(f"{folder_name}/domains/domains*.csv")
print(csv_files)
for i, csv_file in enumerate(csv_files):
    run_crawl(csv_file, folder_name, i)

['data/us_mails_more/domains/domains_27.csv', 'data/us_mails_more/domains/domains_0.csv', 'data/us_mails_more/domains/domains_5.csv', 'data/us_mails_more/domains/domains_22.csv', 'data/us_mails_more/domains/domains_6.csv', 'data/us_mails_more/domains/domains_12.csv', 'data/us_mails_more/domains/domains_4.csv', 'data/us_mails_more/domains/domains_21.csv', 'data/us_mails_more/domains/domains_10.csv', 'data/us_mails_more/domains/domains_25.csv', 'data/us_mails_more/domains/domains_15.csv', 'data/us_mails_more/domains/domains_1.csv', 'data/us_mails_more/domains/domains_2.csv', 'data/us_mails_more/domains/domains_8.csv', 'data/us_mails_more/domains/domains_9.csv', 'data/us_mails_more/domains/domains_7.csv', 'data/us_mails_more/domains/domains_18.csv', 'data/us_mails_more/domains/domains_17.csv', 'data/us_mails_more/domains/domains_13.csv', 'data/us_mails_more/domains/domains_23.csv', 'data/us_mails_more/domains/domains_19.csv', 'data/us_mails_more/domains/domains_11.csv', 'data/us_mails_mor

In [3]:
csv_files = glob.glob(f"{folder_name}/emails/emails*.csv")
len(csv_files)

30

In [4]:
csv_file_list = []
for f in csv_files:
    print(f)
    csv_file_list.append(pd.read_csv(f))
    
df = pd.concat(csv_file_list, ignore_index=True)
df

data/us_mails_more/emails/emails_4.csv
data/us_mails_more/emails/emails_28.csv
data/us_mails_more/emails/emails_17.csv
data/us_mails_more/emails/emails_3.csv
data/us_mails_more/emails/emails_5.csv
data/us_mails_more/emails/emails_11.csv
data/us_mails_more/emails/emails_26.csv
data/us_mails_more/emails/emails_14.csv
data/us_mails_more/emails/emails_27.csv
data/us_mails_more/emails/emails_16.csv
data/us_mails_more/emails/emails_6.csv
data/us_mails_more/emails/emails_8.csv
data/us_mails_more/emails/emails_29.csv
data/us_mails_more/emails/emails_13.csv
data/us_mails_more/emails/emails_18.csv
data/us_mails_more/emails/emails_0.csv
data/us_mails_more/emails/emails_25.csv
data/us_mails_more/emails/emails_1.csv
data/us_mails_more/emails/emails_23.csv
data/us_mails_more/emails/emails_10.csv
data/us_mails_more/emails/emails_21.csv
data/us_mails_more/emails/emails_24.csv
data/us_mails_more/emails/emails_12.csv
data/us_mails_more/emails/emails_15.csv
data/us_mails_more/emails/emails_22.csv
data/us

,company_name,domain,origin_url,url,emails
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com
1,Tekulus,www.tekulus.com,http://www.tekulus.com,https://www.tekulus.com/,NaN
2,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com/About.php,admin@computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com
4,Domino Computer & Networking,www.computerrepairservicevallejo.com,http://www.computerrepairservicevallejo.com,https://www.computerrepairservicevallejo.com/,NaN
...,...,...,...,...,...
136431,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/about-us/newsroom/ukg-acqu...,NaN
136432,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/about-us/newsroom,NaN
136433,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/about-us/careers/early-car...,NaN
136434,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/about-us/ukg-ambassadors,NaN


In [5]:
df.isnull().sum()

company_name        0
domain              0
origin_url          0
url                 0
emails          68747
dtype: int64

In [6]:
df = df.dropna(subset=["emails"])
df

,company_name,domain,origin_url,url,emails
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com
2,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com/About.php,admin@computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com
5,Prospects DM,www.prospectsdm.com,https://prospectsdm.com/,https://prospectsdm.com/about-us/the-future/,info@prospectsdm.com
6,Online Business Automator,www.onlinebusinessautomator.com,https://www.onlinebusinessautomator.com,https://onlinebusinessautomator.com/,sophie@sophiekessner.com
...,...,...,...,...,...
136423,easeIT LLC,www.goeaseit.com,https://www.goeaseit.com,https://goeaseit.com/,it@goeaseit.atlassian.net
136424,easeIT LLC,www.goeaseit.com,https://goeaseit.com/,https://goeaseit.com/contact-us,"support@goeaseit.com,it@goeaseit.atlassian.net"
136426,Next Level PCs | Custom Built Personal Compute...,next-level-pcs-custom-built-computers-vallejo....,https://next-level-pcs-custom-built-computers-...,https://www.google.com/maps/place//data=!4m2!3...,robert@broofa.com
136427,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/contact,elqadmin@ukg.com


In [7]:
df.shape

(67689, 5)

In [8]:
df.nunique()

company_name    29779
domain          29665
origin_url      38942
url             66680
emails          35971
dtype: int64

In [9]:
df = df.assign(emails=df.emails.str.split(",")).explode("emails")
df

,company_name,domain,origin_url,url,emails
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com
2,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com/About.php,admin@computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com
5,Prospects DM,www.prospectsdm.com,https://prospectsdm.com/,https://prospectsdm.com/about-us/the-future/,info@prospectsdm.com
6,Online Business Automator,www.onlinebusinessautomator.com,https://www.onlinebusinessautomator.com,https://onlinebusinessautomator.com/,sophie@sophiekessner.com
...,...,...,...,...,...
136424,easeIT LLC,www.goeaseit.com,https://goeaseit.com/,https://goeaseit.com/contact-us,support@goeaseit.com
136424,easeIT LLC,www.goeaseit.com,https://goeaseit.com/,https://goeaseit.com/contact-us,it@goeaseit.atlassian.net
136426,Next Level PCs | Custom Built Personal Compute...,next-level-pcs-custom-built-computers-vallejo....,https://next-level-pcs-custom-built-computers-...,https://www.google.com/maps/place//data=!4m2!3...,robert@broofa.com
136427,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/contact,elqadmin@ukg.com


In [10]:
df['emails'] = df['emails'].str.strip().str.lower()
df

,company_name,domain,origin_url,url,emails
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com
2,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com/About.php,admin@computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com
5,Prospects DM,www.prospectsdm.com,https://prospectsdm.com/,https://prospectsdm.com/about-us/the-future/,info@prospectsdm.com
6,Online Business Automator,www.onlinebusinessautomator.com,https://www.onlinebusinessautomator.com,https://onlinebusinessautomator.com/,sophie@sophiekessner.com
...,...,...,...,...,...
136424,easeIT LLC,www.goeaseit.com,https://goeaseit.com/,https://goeaseit.com/contact-us,support@goeaseit.com
136424,easeIT LLC,www.goeaseit.com,https://goeaseit.com/,https://goeaseit.com/contact-us,it@goeaseit.atlassian.net
136426,Next Level PCs | Custom Built Personal Compute...,next-level-pcs-custom-built-computers-vallejo....,https://next-level-pcs-custom-built-computers-...,https://www.google.com/maps/place//data=!4m2!3...,robert@broofa.com
136427,"NOVAtime Technology, Inc.",www.novatime.com,https://www.ukg.com/ascentis,https://www.ukg.com/contact,elqadmin@ukg.com


In [11]:
df = df.drop_duplicates(subset=["emails"])
df

,company_name,domain,origin_url,url,emails
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com
6,Online Business Automator,www.onlinebusinessautomator.com,https://www.onlinebusinessautomator.com,https://onlinebusinessautomator.com/,sophie@sophiekessner.com
9,North Coast Tech Support,northcoasttechsupport.com,https://northcoasttechsupport.com,https://northcoasttechsupport.com,support@northcoasttechsupport.com
10,Genentech Inc.,www.gene.com,https://www.gene.com,https://www.gene.com,alpine@v2.x.x
...,...,...,...,...,...
136418,"Ronen Solutions, LLC.",ronensolutions.com,https://ronensolutions.com,https://ronensolutions.com,info@ronensolutions.com
136419,"PC Service Center - Business IT Support, Manag...",www.pc-servicecenter.com,http://www.pc-servicecenter.com,http://www.pc-servicecenter.com/contact.html,hk@mysite.com
136419,"PC Service Center - Business IT Support, Manag...",www.pc-servicecenter.com,http://www.pc-servicecenter.com,http://www.pc-servicecenter.com/contact.html,info@pc-servicecenter.com
136423,easeIT LLC,www.goeaseit.com,https://www.goeaseit.com,https://goeaseit.com/,it@goeaseit.atlassian.net


In [12]:
import tldextract

def get_domain_without_subdomain(url):
    extracted = tldextract.extract(url)
    return "{}.{}".format(extracted.domain, extracted.suffix)

df['domain_tld'] = df['url'].apply(get_domain_without_subdomain)
df

/tmp/ipykernel_13665/1427031664.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain_tld'] = df['url'].apply(get_domain_without_subdomain)


,company_name,domain,origin_url,url,emails,domain_tld
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com,computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com,prospectsdm.com
6,Online Business Automator,www.onlinebusinessautomator.com,https://www.onlinebusinessautomator.com,https://onlinebusinessautomator.com/,sophie@sophiekessner.com,onlinebusinessautomator.com
9,North Coast Tech Support,northcoasttechsupport.com,https://northcoasttechsupport.com,https://northcoasttechsupport.com,support@northcoasttechsupport.com,northcoasttechsupport.com
10,Genentech Inc.,www.gene.com,https://www.gene.com,https://www.gene.com,alpine@v2.x.x,gene.com
...,...,...,...,...,...,...
136418,"Ronen Solutions, LLC.",ronensolutions.com,https://ronensolutions.com,https://ronensolutions.com,info@ronensolutions.com,ronensolutions.com
136419,"PC Service Center - Business IT Support, Manag...",www.pc-servicecenter.com,http://www.pc-servicecenter.com,http://www.pc-servicecenter.com/contact.html,hk@mysite.com,pc-servicecenter.com
136419,"PC Service Center - Business IT Support, Manag...",www.pc-servicecenter.com,http://www.pc-servicecenter.com,http://www.pc-servicecenter.com/contact.html,info@pc-servicecenter.com,pc-servicecenter.com
136423,easeIT LLC,www.goeaseit.com,https://www.goeaseit.com,https://goeaseit.com/,it@goeaseit.atlassian.net,goeaseit.com


In [13]:
# get the rows where the emails column contains the value of domain column
df_filtered = df[df.apply(lambda row: row['domain_tld'] in row['emails'], axis=1)]
df_filtered

,company_name,domain,origin_url,url,emails,domain_tld
0,Computers Corner,www.computerscorner.com,http://www.computerscorner.com,http://www.computerscorner.com,admin@computerscorner.com,computerscorner.com
3,Prospects DM,www.prospectsdm.com,http://www.prospectsdm.com,https://prospectsdm.com/,info@prospectsdm.com,prospectsdm.com
9,North Coast Tech Support,northcoasttechsupport.com,https://northcoasttechsupport.com,https://northcoasttechsupport.com,support@northcoasttechsupport.com,northcoasttechsupport.com
16,RevuEase,reputationmgmt.revuease.com,https://reputationmgmt.revuease.com,https://reputationmgmt.revuease.com,owen@revuease.com,revuease.com
34,Prospects DM,www.prospectsdm.com,https://prospectsdm.com/,https://prospectsdm.com/about-us/privacy-policy/,privacyofficer@prospectsdm.com,prospectsdm.com
...,...,...,...,...,...,...
136379,Resource Computer Solutions,www.resourcecomputer.com,https://www.resourcecomputer.com,https://www.resourcecomputer.com,info@resourcecomputer.com,resourcecomputer.com
136391,Taxware Systems,www.taxwaresystems.com,https://www.taxwaresystems.com,https://www.taxwaresystems.com,support@taxwaresystems.com,taxwaresystems.com
136418,"Ronen Solutions, LLC.",ronensolutions.com,https://ronensolutions.com,https://ronensolutions.com,info@ronensolutions.com,ronensolutions.com
136419,"PC Service Center - Business IT Support, Manag...",www.pc-servicecenter.com,http://www.pc-servicecenter.com,http://www.pc-servicecenter.com/contact.html,info@pc-servicecenter.com,pc-servicecenter.com


In [14]:
df_rest = df[~df.apply(lambda row: row['domain_tld'] in row['emails'], axis=1)]
df_rest

,company_name,domain,origin_url,url,emails,domain_tld
6,Online Business Automator,www.onlinebusinessautomator.com,https://www.onlinebusinessautomator.com,https://onlinebusinessautomator.com/,sophie@sophiekessner.com,onlinebusinessautomator.com
10,Genentech Inc.,www.gene.com,https://www.gene.com,https://www.gene.com,alpine@v2.x.x,gene.com
27,Oliver Web Guy LLC,www.oliverwebguy.com,http://www.oliverwebguy.com,https://www.oliverwebguy.com/,levensonoliver@gmail.com,oliverwebguy.com
67,Genentech Inc.,www.gene.com,https://www.gene.com,https://www.roche.com/investors/us-investors/,rocheirna@gene.com,roche.com
67,Genentech Inc.,www.gene.com,https://www.gene.com,https://www.roche.com/investors/us-investors/,jpmorgan.adr@equiniti.com,roche.com
...,...,...,...,...,...,...
136333,Thinkcar Tech Inc.,www.thinkcarus.com,https://www.thinkcarus.com,https://thinkcarus.com/,product@company.com,thinkcarus.com
136344,Telemanagement Pros Inc. DBA TMP Business Solu...,www.telemanagementpros.com,https://telemanagementpros.com/,https://telemanagementpros.com/contact-us/,info@tmpcomm.com,telemanagementpros.com
136358,Nuvision,www.nuvision.com,https://www.nuvision.com/,https://www.nuvision.com/contact,nuvisioncs@outlook.com,nuvision.com
136419,"PC Service Center - Business IT Support, Manag...",www.pc-servicecenter.com,http://www.pc-servicecenter.com,http://www.pc-servicecenter.com/contact.html,hk@mysite.com,pc-servicecenter.com


In [15]:
df_filtered.nunique()

company_name    21236
domain          21236
origin_url      22676
url             25449
emails          49516
domain_tld      21259
dtype: int64

In [57]:
from urllib.parse import urlparse
# df_original = pd.read_csv("data/us_mails/IT Companies.csv")
# df_original['domain'] = df_original['Website'].apply(urlparse).apply(lambda x: x.netloc)
# df_original.drop_duplicates(subset=['domain'], inplace=True)
# df_merged = pd.merge(df_filtered, df_original[['domain', 'Name']], on='domain', how='inner')
# df_merged

,company_name,domain,origin_url,url,emails,domain_tld,Name
0,I Repair Broken Laptops,irepairbrokenlaptops.com,https://irepairbrokenlaptops.com,https://irepairbrokenlaptops.com,irepairbrokenlaptops.com@gmail.com,irepairbrokenlaptops.com,I Repair Broken Laptops
1,Phoenix Design Co,www.phoenixdesign.com,https://www.phoenixdesign.com,https://www.phoenixdesign.com,fcz@phoenixdesign.com,phoenixdesign.com,Phoenix Design Co
2,NaN,www.phoenixdesign.com,https://www.phoenixdesign.com,https://www.phoenixdesign.com/about/company,bsi@phoenixdesign.com,phoenixdesign.com,Phoenix Design Co
3,NaN,www.phoenixdesign.com,https://www.phoenixdesign.com,https://www.phoenixdesign.com/about/team,lli@phoenixdesign.com,phoenixdesign.com,Phoenix Design Co
4,NaN,www.phoenixdesign.com,https://www.phoenixdesign.com,https://www.phoenixdesign.com/about/team,dkn@phoenixdesign.com,phoenixdesign.com,Phoenix Design Co
...,...,...,...,...,...,...,...
21687,NaN,www.lightcrest.com,http://www.lightcrest.com,https://www.lightcrest.com/contact_upwork/,sales@lightcrest.com,lightcrest.com,Lightcrest
21688,NaN,www.lightcrest.com,http://www.lightcrest.com,https://www.lightcrest.com/contact_upwork/,partners@lightcrest.com,lightcrest.com,Lightcrest
21689,NaN,www.lightcrest.com,http://www.lightcrest.com,https://www.lightcrest.com/contact_upwork/,support@lightcrest.com,lightcrest.com,Lightcrest
21690,NaN,www.lightcrest.com,http://www.lightcrest.com,https://www.lightcrest.com/about-us_upwork/,contact@www.lightcrest.com,lightcrest.com,Lightcrest


In [58]:
df_merged.drop(columns=['company_name', 'origin_url','domain_tld'], inplace=True)
df_merged = df_merged[['Name',"url", 'domain', 'emails']]
df_merged

,Name,url,domain,emails
0,I Repair Broken Laptops,https://irepairbrokenlaptops.com,irepairbrokenlaptops.com,irepairbrokenlaptops.com@gmail.com
1,Phoenix Design Co,https://www.phoenixdesign.com,www.phoenixdesign.com,fcz@phoenixdesign.com
2,Phoenix Design Co,https://www.phoenixdesign.com/about/company,www.phoenixdesign.com,bsi@phoenixdesign.com
3,Phoenix Design Co,https://www.phoenixdesign.com/about/team,www.phoenixdesign.com,lli@phoenixdesign.com
4,Phoenix Design Co,https://www.phoenixdesign.com/about/team,www.phoenixdesign.com,dkn@phoenixdesign.com
...,...,...,...,...
21687,Lightcrest,https://www.lightcrest.com/contact_upwork/,www.lightcrest.com,sales@lightcrest.com
21688,Lightcrest,https://www.lightcrest.com/contact_upwork/,www.lightcrest.com,partners@lightcrest.com
21689,Lightcrest,https://www.lightcrest.com/contact_upwork/,www.lightcrest.com,support@lightcrest.com
21690,Lightcrest,https://www.lightcrest.com/about-us_upwork/,www.lightcrest.com,contact@www.lightcrest.com


In [59]:
df_merged.isnull().sum()

Name      0
url       0
domain    0
emails    0
dtype: int64

In [60]:
# reset index to "domain" column
df_merged = df_merged.set_index("domain")
df_merged

,Name,url,emails
domain,,,
irepairbrokenlaptops.com,I Repair Broken Laptops,https://irepairbrokenlaptops.com,irepairbrokenlaptops.com@gmail.com
www.phoenixdesign.com,Phoenix Design Co,https://www.phoenixdesign.com,fcz@phoenixdesign.com
www.phoenixdesign.com,Phoenix Design Co,https://www.phoenixdesign.com/about/company,bsi@phoenixdesign.com
www.phoenixdesign.com,Phoenix Design Co,https://www.phoenixdesign.com/about/team,lli@phoenixdesign.com
www.phoenixdesign.com,Phoenix Design Co,https://www.phoenixdesign.com/about/team,dkn@phoenixdesign.com
...,...,...,...
www.lightcrest.com,Lightcrest,https://www.lightcrest.com/contact_upwork/,sales@lightcrest.com
www.lightcrest.com,Lightcrest,https://www.lightcrest.com/contact_upwork/,partners@lightcrest.com
www.lightcrest.com,Lightcrest,https://www.lightcrest.com/contact_upwork/,support@lightcrest.com


In [61]:
df_merged.to_csv(f"{folder_name}/us_clean.csv")

In [62]:
df_merged.drop(columns=['url'], inplace=True)
df_merged

,Name,emails
domain,,
irepairbrokenlaptops.com,I Repair Broken Laptops,irepairbrokenlaptops.com@gmail.com
www.phoenixdesign.com,Phoenix Design Co,fcz@phoenixdesign.com
www.phoenixdesign.com,Phoenix Design Co,bsi@phoenixdesign.com
www.phoenixdesign.com,Phoenix Design Co,lli@phoenixdesign.com
www.phoenixdesign.com,Phoenix Design Co,dkn@phoenixdesign.com
...,...,...
www.lightcrest.com,Lightcrest,sales@lightcrest.com
www.lightcrest.com,Lightcrest,partners@lightcrest.com
www.lightcrest.com,Lightcrest,support@lightcrest.com


In [63]:
# implode emails with same domain column. Make it comma separated mails in email columns.
df_merged = df_merged.groupby('domain').agg({
    'emails' : lambda x: ','.join(list(x.unique())),
    'Name' : lambda x: x.unique()[0]
    
}).reset_index()

df_merged

,domain,emails,Name
0,01infotech.com,salhaider@01infotech.com,"01 InfoTech, Inc."
1,10-8systems.com,"contact@10-8systems.com,support@10-8systems.com",10-8 Systems
2,101digital.com,hello@101digital.com,101 Digital
3,10tier.com,"info@10tier.com,westchester@10tier.com",NYC SEO Company: 10Tier.com
4,11marketing.com,hello@11marketing.com,11 Marketing + Design
...,...,...,...
10421,zonapact.com,"support@zonapact.com.,support@zonapact.com",Zonapact
10422,zorang.com,"marketing@zorang.com,sales@zorang.com",Zorang
10423,zorgtek.com,mail@zorgtek.com,ZorgTek
10424,zverse.com,info@zverse.com,"ZVerse, Inc."


In [64]:
df_merged = df_merged[['Name', 'domain', 'emails']]
df_merged

,Name,domain,emails
0,"01 InfoTech, Inc.",01infotech.com,salhaider@01infotech.com
1,10-8 Systems,10-8systems.com,"contact@10-8systems.com,support@10-8systems.com"
2,101 Digital,101digital.com,hello@101digital.com
3,NYC SEO Company: 10Tier.com,10tier.com,"info@10tier.com,westchester@10tier.com"
4,11 Marketing + Design,11marketing.com,hello@11marketing.com
...,...,...,...
10421,Zonapact,zonapact.com,"support@zonapact.com.,support@zonapact.com"
10422,Zorang,zorang.com,"marketing@zorang.com,sales@zorang.com"
10423,ZorgTek,zorgtek.com,mail@zorgtek.com
10424,"ZVerse, Inc.",zverse.com,info@zverse.com


In [65]:
df_merged_2 = pd.merge(df_original, df_merged[['domain','emails']], on='domain', how='left')
df_merged_2['emails'] = df_merged_2['emails'].fillna('')
df_merged_2

,Name,Website,domain,emails
0,Jafton | Mobile App Development New York,https://www.jafton.com/top-mobile-app-developm...,www.jafton.com,"hello@jafton.com,job@jafton.com"
1,RisingMax - IT Consulting Company,https://risingmax.com/,risingmax.com,idea@risingmax.com
2,IN NYC IT,http://innycit.com/,innycit.com,support@innycit.com
3,Exigent Technologies | NYC Managed IT Services,https://www.exigent.net/contact/office-locatio...,www.exigent.net,
4,Agicent App Company NYC,https://www.agicent.com/,www.agicent.com,"sales@agicent.com.,sales@agicent.com"
...,...,...,...,...
21873,The Village at Camelot,https://www.thevillageatcamelot.com/,www.thevillageatcamelot.com,
21874,Estrella Vista by Express Homes,https://www.drhorton.com/arizona/phoenix/bucke...,www.drhorton.com,
21875,Roosevelt Irrigation District,http://www.rooseveltirrigation.org/,www.rooseveltirrigation.org,
21876,Ramirez's Discount Garage Doors LLC,https://ramirezdiscountgaragedoorsaz.com/,ramirezdiscountgaragedoorsaz.com,


In [66]:
df_merged_2.nunique()

Name       21878
Website    21878
domain     21878
emails     10427
dtype: int64

In [67]:
df_original.nunique()

Name       21878
Website    21878
domain     21878
dtype: int64

In [41]:
df_merged_2.to_csv(f"{folder_name}/us_mails_all.csv")